In [38]:
import random

def strip_punctuation(str):
    return str.strip('.;,-––“’”":?—‘!()_')

# Wir bauen ein Sprachmodell

Im letzten Notebook haben wir uns angeschaut, wie wir aus einem Wortpaar-Modell Text generieren können.
Da nur ein einziges Wort als Kontext verwendet wurde, macht der generierte Text noch nicht so viel Sinn.

Schauen wir mal, ob wir den generierten Text verbessern können, wenn wir mehr Kontext hinzufügen.

In [39]:
reader = open('data/der-gestiefelte-kater.txt')
successor_map = {}
window = []

for line in reader:
  for word in line.split():
    clean_word = strip_punctuation(word).lower()
    window.append(clean_word)
  
    if len(window) == 2:
      key = window[0]
      value = window[1]
      if key in successor_map:
        successor_map[key].append(value)
      else:
        successor_map[key] = [value]
      window.pop(0)

print(successor_map['der'])

['gestiefelte', 'gute', 'dicke', 'dünne', 'kater', 'hinze', 'kater', 'hans', 'hans', 'hinze', 'hans', 'angst', 'hans', 'hinze', 'hans', 'hinze', 'gerade', 'kater', 'sprechen', 'wurde', 'napoleon', 'ist', 'kater', 'schuster', 'fußbank', 'könig', 'hatte', 'könig', 'weg', 'brust', 'mund', 'alberne', 'troddel', 'koch', 'säckelmeister', 'herr', 'herr', 'hinze', 'hinze', 'herr', 'hinze', 'hans', 'dicke', 'dünne', 'hinze', 'uebung', 'mir', 'könig', 'zufall', 'hinze', 'küche', 'kutscher', 'milz', 'alte', 'dicke', 'wind', 'hans', 'hinz', 'hinze', 'hans', 'vorreiter', 'hinz', 'goldene', 'könig', 'mit', 'könig', 'hinze', 'könig', 'hinze', 'junge', 'könig', 'prinzessin', 'könig', 'hinze', 'wald', 'herr', 'zauberer', 'hofwagen', 'könig', 'hinze', 'könig', 'tat', 'hinze-tausendsassa', 'wald', 'fußabstreicher', 'hinz', 'große', 'universität', 'diener', 'zauberer', 'ruhm', 'große', 'eitelkeit', 'große', 'zauberer', 'leistungen', 'damit', 'kater', 'zauberer', 'hinz', 'sprache', 'hinze', 'sprache', 'ele

Dann können wir mithilfe dieses Folgewörter-Nachschlagewerks Text generieren.

In [40]:
word = "der"
for i in range(15):
    print(word, end = ' ')
    word = random.choice(successor_map[word])

der hinz ihm der zauberer aus schlangen krochen drum reden wir kommen nimmer von carabas 

Können wir die Ergebnisse verbessern, wenn wir 2 aufeinanderfolgende Wörter verwenden?

In [41]:
reader = open('data/der-gestiefelte-kater.txt')

def generate_successors(lines, contextSize, strip = True, lowercase = True):
    
    successor_map = {}
    window = []
    if (contextSize < 1):
        return successor_map

    for line in lines:
        for word in line.split():
            clean_word = word
            if strip:
                clean_word = strip_punctuation(clean_word)
            if lowercase:
                clean_word = clean_word.lower()
            window.append(clean_word)
  
            if len(window) == contextSize + 1:
                key = tuple(window[0:contextSize])
                value = window[contextSize]
                if key not in successor_map:
                    successor_map[key] = [value]
                else:
                    successor_map[key].append(value) 
                window.pop(0)
    
    return successor_map

successor_map = generate_successors(reader, 2)
print(successor_map[("der", "kater")])

['also', 'verstände', 'auf', 'die', 'gesagt']


Wir können jetzt ein Dictionary aufbauen, was 2 oder mehr vorhergegangene Wörter als Kontext verwendet. 
Was schonmal auffällt ist, dass die Auswahl an Folgeworten, die nach "Der Kater" kommen, sehr viel kleiner ist. Das ist gut. Es macht unter Umständen mehr Sinn :).

In [36]:
# random.seed(3)
word1 = "die"
word2 = "stiefel"

for i in range(15):
  print (word1, end=" ")
  successors = successor_map[(word1, word2)]
  word3 = random.choice(successors)
  word1 = word2
  word2 = word3

die stiefel zu wischen denn dann spiegelten sie immer noch blitzblanker wie ein menschenniesen und 

Mit einem Kontext von zwei Folgewörtern ist der generierte Text schon ein wenig besser. Aber so richtig Sinn macht er noch nicht wirklich. Was ist das Problem?

- Die Satzzeichen fehlen
- Nach ein paar Wörtern macht der Text keinen Sinn mehr

In [47]:
reader = open('data/der-gestiefelte-kater.txt')
successor_map = generate_successors(reader, 2, False, False)

word1 = "Der"
word2 = "gestiefelte"

for i in range(30):
  print (word1, end=" ")
  successors = successor_map[(word1, word2)]
  word3 = random.choice(successors)
  word1 = word2
  word2 = word3

Der gestiefelte Kater Ja, nun war der gute Vater tot, und der Hinze, "o großmächtiger Zauberer, reicht deine Güte wirklich so weit, daß du mir – oder ich lasse dich 

Mit dem Einbeziehen der Satzzeichen wird es etwas besser, aber nicht viel. Was könnte helfen, sinnhafteren Text zu generieren?

Genau. Kontext vergrößern. Aber da gibt es leider einen Haken, den wir bereits bei einem Kontext von 2 feststellen können. 

- je größer der Kontext, desto kleiner wird die Auswahlmöglichkeit möglicher Folgeworte, was dazu führen kann, dass immer das gleiche rauskommt
- je größer der Kontext, desto schwieriger wird es zu raten, welche Worte in unserem Modell aufeinander folgen. ('Die', 'Stiefel') kommt beispielsweise nicht vor.
- das heißt: je größer der Kontext, desto mehr Trainingdaten brauchen wir.